## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [40]:
# When using Colab, make sure you run this instruction beforehand

!pip install --upgrade cffi==1.15.0
import importlib
import cffi
import numpy as np
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import mip

Project made by:
Tommaso Aiello |
Evelina Arnaudova |
Brian Shehara Appuhamy

## Sets

In [42]:
#number of farms
farms=5
F = [i for i in range(farms)]
#number of quarters
quarters = 8
Q = [j for j in range(quarters)]

## Parameters

In [43]:
#quarter on column, farm on rows
corn_availability =[[700.0, 1500.0 ,700.0 , 0.0 , 0.0 , 700.0 , 1500.0 , 0.0 ],
                    [1350.0 , 0.0 , 450.0 , 0.0 , 1350.0 , 0.0 , 450.0 , 0.0 ],
                    [0.0,1500.0,1500.0, 0.0 , 0.0 , 1500.0 , 1500.0 , 0.0 ],
                    [820.0 ,1560.0 , 820.0 , 0.0, 820.0 , 1560.0 , 820.0 , 0.0],
                    [0.0 , 680.0 , 1080.0 ,0.0 ,0.0 ,680.0,1080.0,0.0]]

#purchase price per farm
purchase_price = [0.2,0.18,0.19,0.21,0.23] #i

#storage cost
storage_cost = [0.002,0.012,0.007,0.011,0.015] #i

#dry_matter
dry_matter = [0.15,0.28,0.35,0.37,0.42] #i

#percentage of drymatter
min_percentage = 0.20
max_percentage = 0.40

#energy that a ton of crop produces, in kWh
energy_per_ton = 421.6

#revenue that a kwh generates
market_price = 0.28

#energy that can be produced in a quarter at most, in kWh
max_energy_quarter = 1950000.0

#max ton of crop that can be stored in silos
silos_capacity = 500.0


## Model

In [44]:
m = mip.Model()

## Variables

In [45]:
# A quantity that we buy for each farm i for each quarter j 
quantity = [[m.add_var(var_type=mip.CONTINUOUS) for j in Q] for i in F]

In [46]:
# A variable to count the tons of crop for each farm i in quarter j that we have put in the silos
corn_in_sillos = [[m.add_var(var_type= mip.CONTINUOUS) for j in Q] for i in F]

In [47]:
# Tons of crop from farm i in quarter j we burn to produce energy 
burnt_quantity = [[m.add_var(var_type=mip.CONTINUOUS) for j in Q] for i in F]

## Objective Function

In [48]:
# O.F. = max(The total revenue - the total cost to purchase the crop - the total storage cost)
m.objective = mip.maximize(
    mip.xsum(mip.xsum(burnt_quantity[i][j] * energy_per_ton * market_price for i in F) for j in Q)  # revenue
    -mip.xsum(mip.xsum(quantity[i][j] * purchase_price[i] for i in F) for j in Q)                   # purchase cost
    -mip.xsum(mip.xsum(corn_in_sillos[i][j] * storage_cost[i] for i in F) for j in Q)               # sillos's cost
)

## Constraints

In [49]:
# Non negativity constraints
for i in F:
  for j in Q:
    m.add_constr(quantity[i][j] >= 0.0)
    m.add_constr(corn_in_sillos[i][j] >= 0.0)
    m.add_constr(burnt_quantity[i][j] >= 0.0)

In [50]:
# Balancing constrain

# What you buy - what you will have in the storage at the end of quarter = to tons that were burnt
# What you buy + what you had in storage - what you will have in the storage at the end of quarter = to tons that were burnt
# Done this for each farm i and for each quarters j

for i in F:
  m.add_constr( (quantity[i][0]) - corn_in_sillos[i][0] == (burnt_quantity[i][0]))
for j in Q[1:]: 
  for i in F:
    m.add_constr( (quantity[i][j]) - corn_in_sillos[i][j] + corn_in_sillos[i][j-1] == (burnt_quantity[i][j]))



In [51]:
# Silos capacity
for j in Q:
    m.add_constr(mip.xsum(corn_in_sillos[i][j] for i in F) <= silos_capacity )

In [52]:
# Farm capacity: corn availability for each farm
for i in F:
  for j in Q:
    m.add_constr(quantity[i][j] <= corn_availability[i][j])

In [53]:
# Max energy that you can produce in each quarter
for j in Q:
  m.add_constr(mip.xsum(burnt_quantity[i][j] for i in F)*energy_per_ton <= max_energy_quarter)

In [54]:
# Dry matter constraint
for j in Q:
  total_burned = mip.xsum(burnt_quantity[i][j] for i in F)
  total_dry_matter = mip.xsum((burnt_quantity[i][j]*dry_matter[i]) for i in F )
  m.add_constr((total_burned*min_percentage) <= total_dry_matter)
  m.add_constr((total_burned*max_percentage) >= total_dry_matter)

## ACTUAL OPTIMIZATION

In [55]:
status = m.optimize()
print('After planning the supply and the invenotry, the maximized profit would be: ')
print(round(m.objective_value,4))

After planning the supply and the invenotry, the maximized profit would be: 
2861373.9254


In [56]:
status == mip.OptimizationStatus.OPTIMAL

True

In [57]:
#BRIEF SUMMARY AND PRINT OF THE RESULTS
for j in Q:
  print('\nQuarter ' +  str(j+1) + '\n')
  energy_produced = 0
  percentage = 0
  tot_burned=0
  dry_burned=0
  money_made=0
  cost_of_crop=0
  cost_of_storage=0
  for i in F:
    energy_produced = energy_produced + (burnt_quantity[i][j].x * energy_per_ton)
    tot_burned = tot_burned + burnt_quantity[i][j].x
    dry_burned = dry_burned + (burnt_quantity[i][j].x * dry_matter[i])
    if tot_burned != 0:
      percentage= dry_burned / tot_burned
    money_made= money_made + (burnt_quantity[i][j].x * energy_per_ton * market_price)
    cost_of_crop = cost_of_crop + (quantity[i][j].x * purchase_price[i])
    cost_of_storage = cost_of_storage + (corn_in_sillos[i][j].x* storage_cost[i])
    print('BUY ' + str(i+1) +': '+ str(round(quantity[i][j].x,3))+ '/' + str(round(corn_availability[i][j],3))  + ' || IN SILOS : '+ str(round(corn_in_sillos[i][j].x,3)) + ' || BURN : ' + str(round(burnt_quantity[i][j].x,3)) )
  print('\nENERGY: '+ str(round(energy_produced,3))+ ' kwh\n')
  print('Percentage of dry matter: '+ str(round(percentage,3))+ '\n')
  print('Money made: ' + str(round(money_made,2)) + '\n' )
  print('Cost of crop: ' + str(round(cost_of_crop,3)) + '\n')
  print('Cost of storage: ' + str(round(cost_of_storage,3)) + '\n')
  print('Earnings: '+ str(round(money_made-cost_of_crop-cost_of_storage,3))+ '\n')



Quarter 1

BUY 1: 700.0/700.0 || IN SILOS : 0.0 || BURN : 700.0
BUY 2: 1350.0/1350.0 || IN SILOS : 0.0 || BURN : 1350.0
BUY 3: 0.0/0.0 || IN SILOS : 0.0 || BURN : 0.0
BUY 4: 820.0/820.0 || IN SILOS : 0.0 || BURN : 820.0
BUY 5: 0.0/0.0 || IN SILOS : 0.0 || BURN : 0.0

ENERGY: 1209992.0 kwh

Percentage of dry matter: 0.274

Money made: 338797.76

Cost of crop: 555.2

Cost of storage: 0.0

Earnings: 338242.56


Quarter 2

BUY 1: 1500.0/1500.0 || IN SILOS : 500.0 || BURN : 1000.0
BUY 2: 0.0/0.0 || IN SILOS : 0.0 || BURN : 0.0
BUY 3: 1500.0/1500.0 || IN SILOS : 0.0 || BURN : 1500.0
BUY 4: 1560.0/1560.0 || IN SILOS : 0.0 || BURN : 1560.0
BUY 5: 565.237/680.0 || IN SILOS : 0.0 || BURN : 565.237

ENERGY: 1950000.0 kwh

Percentage of dry matter: 0.322

Money made: 546000.0

Cost of crop: 1042.605

Cost of storage: 1.0

Earnings: 544956.395


Quarter 3

BUY 1: 700.0/700.0 || IN SILOS : 318.572 || BURN : 881.428
BUY 2: 450.0/450.0 || IN SILOS : 0.0 || BURN : 450.0
BUY 3: 1500.0/1500.0 || IN SILO